# <center>Get CNN model prediction </center>

The 1D CNN model is now deployed as a REST service in Watson Machine Learning and we can predict the churn probability for new data. 

Let's see the deployed model using the python library `WatsonMachineLearningAPIClient`.

In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [2]:
# @hidden_cell
wml_credentials = {
                   "url": "https://us-south.ml.cloud.ibm.com",
                   "apikey":"FQS64OS_Sypionu3f-CYe2-rz_N_KR_leLGySZIewPpe",
                   "instance_id": "aaac2cfb-14ee-4e77-8ee5-88a684128713"
                  }

client = WatsonMachineLearningAPIClient(wml_credentials)

In [ ]:
#Create a WML python client


#wml_credentials = {
#                   "url": "*********",
#                   "apikey":"********",
#                   "instance_id": "*********"
#                  }

Let's see what using `list()` command.

In [3]:
client.repository.list()

------------------------------------  --------------------------  ------------------------  ---------------  -----------------
GUID                                  NAME                        CREATED                   FRAMEWORK        TYPE
0858cadb-bea9-41c3-8a0b-dbf7c2e9003c  CNN_TelefonicaChurn         2019-08-07T17:56:57.825Z  tensorflow-1.13  model
c30bfba1-808b-4249-8bf6-19d93407948d  Keras_CNN_TelefonicaChurn   2019-08-07T17:57:03.151Z  tensorflow-1.13  online deployment
4b20ef2e-1efb-4cb7-b905-1f2b72d167ee  CNN_Prediction_Fuction      2019-08-08T17:24:20.283Z  -                python function
97f884ba-478e-4843-afee-4ef4edfefd4c  CNN_Prediction_Fuction-3.6  2019-08-08T17:24:19.243Z  -                python runtime
------------------------------------  --------------------------  ------------------------  ---------------  -----------------


Now let's see the models deployed using the command `client.deployments.list()`

In [4]:
client.deployments.list()

------------------------------------  -------------------------  ------  --------------  ------------------------  ---------------  -------------
GUID                                  NAME                       TYPE    STATE           CREATED                   FRAMEWORK        ARTIFACT TYPE
c30bfba1-808b-4249-8bf6-19d93407948d  Keras_CNN_TelefonicaChurn  online  DEPLOY_SUCCESS  2019-08-07T17:57:03.151Z  tensorflow-1.13  model
------------------------------------  -------------------------  ------  --------------  ------------------------  ---------------  -------------


We can delete an artifact object using the command `client.repository.delete('*****************')` (replace the asterisks with the corresponding GUID)

In [5]:
#client.repository.delete('**************')

In [6]:
client.deployments.list()

------------------------------------  -------------------------  ------  --------------  ------------------------  ---------------  -------------
GUID                                  NAME                       TYPE    STATE           CREATED                   FRAMEWORK        ARTIFACT TYPE
c30bfba1-808b-4249-8bf6-19d93407948d  Keras_CNN_TelefonicaChurn  online  DEPLOY_SUCCESS  2019-08-07T17:57:03.151Z  tensorflow-1.13  model
------------------------------------  -------------------------  ------  --------------  ------------------------  ---------------  -------------


### Load the Dictionary 

In [7]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_dad6a34ab1754ee2b506e620972cf9fe = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='XWXHrWEjPzqt1Oosn9ytYE4HghzDC6GWqslDj8STHHSz',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

In [8]:
# @hidden_cell
# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about your possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
streaming_body_1 = client_dad6a34ab1754ee2b506e620972cf9fe.get_object(Bucket='telefonicachurn-donotdelete-pr-4oqv8m24vuj4uu', Key='DESCRIPCION VARIABLES IBM V1.xlsx')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(streaming_body_1, "__iter__"): streaming_body_1.__iter__ = types.MethodType( __iter__, streaming_body_1 ) 

col_descrip = pd.read_excel(streaming_body_1, sheet_name="Hoja1")

In [9]:
col_descrip = col_descrip[['VARIABLE', 'DESCRIPCIÓN', 'CAMPOS ANONIMOS']]
col_descrip.head()

,VARIABLE,DESCRIPCIÓN,CAMPOS ANONIMOS
0,ID,ID CLIENTE,ID
1,TIPO_MS,TIPO DE MARKET SHARE,A1
2,TECNOLOGIA,TECNOLOGIA DONDE MÁS NAVEGA,A2
3,TARIFA_BASICA,TARIFA BASICA,A3
4,MB_TOTALES_COBRADOS,CONSUMO DE MB,A4


### Function for preparing the data

In [24]:
import pandas as pd
import numpy
def prepare_data(df):
    df = df[col_descrip['VARIABLE'].values] ## select columns
    df.columns = col_descrip['CAMPOS ANONIMOS'].values
    data = df.iloc[ :,1:116]
    data.iloc[:, 68] = data.iloc[:, 68].fillna("Missing")
    data_dummies = pd.get_dummies(data, drop_first=True)## one-hot encoding
    data_dummies.columns = [['A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'A15', 'A16', 'A17',
                         'A18', 'A19', 'A20', 'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27',
                         'A28', 'A29', 'A30', 'A31', 'A32', 'A33', 'A34', 'A35', 'A36', 'A37',
                         'A38', 'A39', 'A40', 'A41', 'A42', 'A43', 'A44', 'A45', 'A46', 'A47',
                         'A48', 'A49', 'A50', 'A51', 'A52', 'A53', 'A54', 'A55', 'A56', 'A57',
                         'A58', 'A59', 'A60', 'A61', 'A62', 'A63', 'A64', 'A65', 'A66', 'A67',
                         'A69', 'A70', 'A71', 'A72', 'A73', 'A74', 'A75', 'A76', 'A77', 'A78',
                         'A79', 'A80', 'A81', 'A82', 'A83', 'A84', 'A85', 'A86', 'A87', 'A88',
                         'A89', 'A90', 'A91', 'A92', 'A93', 'A94', 'A95', 'A97', 'A98', 'A99',
                         'A100', 'A101', 'A102', 'A103', 'A104', 'A105', 'A106', 'A107', 'A109',
                         'A110', 'A112', 'A113', 'A114', 'A115', 'A116', 'A1_B', 'A1_N', 'A2_3',
                         'A2_4', 'A2_UN', 'A11_N_DC', 'A12_ALCATEL', 'A12_APPLE', 'A12_HUAWEI',
                         'A12_LG', 'A12_MOTOROLA', 'A12_NOKIA', 'A12_OTHER', 'A12_SAMSUNG', 
                         'A12_SIEMENS', 'A12_SONY', 'A13_IOS', 'A13_OTROS', 'A14_SI', 'A68_B',
                         'A108_M', 'A108_N', 'A108_R', 'A111_G', 'A111_M', 'A111_O', 'A111_P', ' A111_T']]
    
    #Select the columns for the CNN
    X_df = data_dummies[['A22', 'A32', 'A35', 'A52', 'A2_UN', 'A49', 'A64', 'A108_N', 'A53',
                                 'A50', 'A27', 'A63', 'A9', 'A33', 'A55', 'A36', 'A18', 'A67', 'A111_P',
                                 'A19', 'A113', 'A44', 'A23', 'A21', 'A68_B', 'A109', 'A20', 'A116',
                                 'A3', 'A2_4', 'A10', 'A62', 'A106', 'A46', 'A105', 'A91', 'A39', 'A1_B',
                                 'A11_N_DC', 'A65', 'A87', 'A12_SAMSUNG', 'A104', 'A110', 'A108_M',
                                 'A16', 'A103', 'A102', 'A12_NOKIA', 'A77', 'A54', 'A83', 'A112', 'A66',
                                 'A98', 'A48', 'A86', 'A28', 'A41', 'A14_SI', 'A108_R', 'A17', 'A30',
                                 'A111_M', 'A13_OTROS', 'A111_O']].values
    X_tensor = X_df.reshape(X_df.shape[0], X_df.shape[1], 1) # convert data to tensor form for the CNN
    return X_tensor
    
    

### Load June Data 

In [21]:
# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials_1 = {
    'IAM_SERVICE_ID': 'iam-ServiceId-90c4b323-ae1c-437f-ad45-c5f7dd631482',
    'IBM_API_KEY_ID': 'XWXHrWEjPzqt1Oosn9ytYE4HghzDC6GWqslDj8STHHSz',
    'ENDPOINT': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.ng.bluemix.net/oidc/token',
    'BUCKET': 'telefonicachurn-donotdelete-pr-4oqv8m24vuj4uu',
    'FILE': 'IBM_TEST_JULIO_v2.txt'
}
client_june_data = ibm_boto3.client(service_name='s3',
    ibm_api_key_id= credentials_1['IBM_API_KEY_ID'],
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about your possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
body = client_june_data.get_object(Bucket=credentials_1['BUCKET'], Key=credentials_1['FILE'])['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body ) 

june_df = pd.read_csv(body)
june_df.head()

,ID,TIPO_MS,TECNOLOGIA,TARIFA_BASICA,MB_TOTALES_COBRADOS,CANTIDAD_MINUTOS,DELTA_UPSELL,DELTA_DOWNSEL,CANTIDAD_SMS,ANTIGUEDAD,...,SUB_SEGMENTO,TOTAL_DEUDA,PAGOS,PROVINCIA,ESTADO_ABONADOSAA,DIAS_VENCIMIENTO0,DIAS_VENCIMIENTO30,DIAS_VENCIMIENTO60,DIAS_VENCIMIENTONO,TARGET
0,1,B,4G,15.00,7973.4291,96.266667,0.0,0.0,6,8,...,M,17.87,2.673755,P,0,1,0,0,0,0
1,2,B,4G,20.99,7442.0808,24.783333,0.0,0.0,2,20,...,N,25.37,1.002365,P,0,1,0,0,0,0
2,3,B,3G,9.99,1070.4361,103.516667,0.0,0.0,2,3,...,R,8.95,1.000000,P,0,1,0,0,0,0
3,4,A,4G,24.99,9268.4092,18.566667,0.0,0.0,3,1,...,N,0.00,1.000000,M,0,0,0,0,1,0
4,5,A,3G,20.99,41.3467,22.250000,0.0,0.0,0,1,...,N,0.00,1.000000,G,0,0,0,0,1,0


In [22]:
june_cnn = prepare_data(june_df)
june_cnn.shape

(750030, 66, 1)

In [35]:
df = june_cnn[0:500]
df.shape

(500, 66, 1)

## Request predictions

First, we need the link to the model, i.e. the scoring url. We can get this url using the following command.

In [37]:
#replace the asterisks with the corresponding GUID
deployment = client.deployments.get_details('**************')
print(deployment)
url_score = client.deployments.get_scoring_url(deployment)
print(url_score)

{'metadata': {'guid': 'c30bfba1-808b-4249-8bf6-19d93407948d', 'url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/aaac2cfb-14ee-4e77-8ee5-88a684128713/deployments/c30bfba1-808b-4249-8bf6-19d93407948d', 'created_at': '2019-08-07T17:57:03.151Z', 'modified_at': '2019-08-07T17:57:03.329Z'}, 'entity': {'runtime_environment': 'None Provided', 'name': 'Keras_CNN_TelefonicaChurn', 'scoring_url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/aaac2cfb-14ee-4e77-8ee5-88a684128713/deployments/c30bfba1-808b-4249-8bf6-19d93407948d/online', 'deployable_asset': {'name': 'CNN_TelefonicaChurn', 'url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/aaac2cfb-14ee-4e77-8ee5-88a684128713/published_models/0858cadb-bea9-41c3-8a0b-dbf7c2e9003c', 'guid': '0858cadb-bea9-41c3-8a0b-dbf7c2e9003c', 'created_at': '2019-08-07T17:57:03.083Z', 'type': 'model'}, 'description': 'Description of deployment', 'status_details': {'status': 'DEPLOY_SUCCESS'}, 'model_type': 'tensorflow-1.13', 'status': 'DEPLOY_

In [25]:
# @hidden_cell
scoring_url = 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/aaac2cfb-14ee-4e77-8ee5-88a684128713/deployments/c30bfba1-808b-4249-8bf6-19d93407948d/online'
print(scoring_url)

https://us-south.ml.cloud.ibm.com/v3/wml_instances/aaac2cfb-14ee-4e77-8ee5-88a684128713/deployments/c30bfba1-808b-4249-8bf6-19d93407948d/online


In [37]:
# @hidden_cell
deployment = client.deployments.get_details('c30bfba1-808b-4249-8bf6-19d93407948d')
print(deployment)
url_score = client.deployments.get_scoring_url(deployment)
print(url_score)

{'metadata': {'guid': 'c30bfba1-808b-4249-8bf6-19d93407948d', 'url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/aaac2cfb-14ee-4e77-8ee5-88a684128713/deployments/c30bfba1-808b-4249-8bf6-19d93407948d', 'created_at': '2019-08-07T17:57:03.151Z', 'modified_at': '2019-08-07T17:57:03.329Z'}, 'entity': {'runtime_environment': 'None Provided', 'name': 'Keras_CNN_TelefonicaChurn', 'scoring_url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/aaac2cfb-14ee-4e77-8ee5-88a684128713/deployments/c30bfba1-808b-4249-8bf6-19d93407948d/online', 'deployable_asset': {'name': 'CNN_TelefonicaChurn', 'url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/aaac2cfb-14ee-4e77-8ee5-88a684128713/published_models/0858cadb-bea9-41c3-8a0b-dbf7c2e9003c', 'guid': '0858cadb-bea9-41c3-8a0b-dbf7c2e9003c', 'created_at': '2019-08-07T17:57:03.083Z', 'type': 'model'}, 'description': 'Description of deployment', 'status_details': {'status': 'DEPLOY_SUCCESS'}, 'model_type': 'tensorflow-1.13', 'status': 'DEPLOY_

To get predictions from the deployed model, the input data must be in the following json format:

In [36]:
payload_scoring = {'values': df.tolist()} #Note that june_cnn is a tensor

Use the following code to get the prediction. You will need the scoring url and the input data in json format.

In [39]:
import json
response_scoring = client.deployments.score(url_score, payload_scoring)
print(json.dumps(response_scoring, indent=2))

{
  "fields": [
    "prediction",
    "prediction_classes",
    "probability"
  ],
  "values": [
    [
      [
        0.5053015351295471
      ],
      [
        1
      ],
      [
        0.5053015351295471
      ]
    ],
    [
      [
        0.6010534167289734
      ],
      [
        1
      ],
      [
        0.6010534167289734
      ]
    ],
    [
      [
        0.3206188976764679
      ],
      [
        0
      ],
      [
        0.3206188976764679
      ]
    ],
    [
      [
        0.3804580569267273
      ],
      [
        0
      ],
      [
        0.3804580569267273
      ]
    ],
    [
      [
        0.4834129512310028
      ],
      [
        0
      ],
      [
        0.4834129512310028
      ]
    ],
    [
      [
        0.2281648814678192
      ],
      [
        0
      ],
      [
        0.2281648814678192
      ]
    ],
    [
      [
        0.6233150959014893
      ],
      [
        1
      ],
      [
        0.6233150959014893
      ]
    ],
    [
      [
